In [17]:
# Build base code for making requests to OpenAI API
import requests
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")
model = "claude-sonnet-4-20250514"
url = "https://api.laozhang.ai/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": api_key
}

def add_system_message(messages, content):
    messages.append({"role": "system", "content": content})

def add_user_message(messages, content):
    messages.append({"role": "user", "content": content})

def add_assistant_message(messages, content):
    messages.append({"role": "assistant", "content": content})

def chat(messages,temperature=None,stop_sequences=[]):
    params = {
        "model": model,
        "stream": False,
        "messages": messages,
        "stop": stop_sequences,
        "max_tokens": 1024,
    }
    if temperature:
        params["temperature"] = temperature

    response = requests.post(url, headers=headers, json=params)
    try: response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
        print(f"Response content: {response.text}")
        raise

    answer = response.json()['choices'][0]['message']['content']
    return answer

In [3]:
system = """
    You are a patient math tutor.
    Do not directly give the answer to the student.
    Instead, guide them to a solution step by step.
"""

messages = []
add_system_message(messages, system)
add_user_message(messages, "What is answer x of the equation 3x+8=20?")
answer = chat(messages)
answer


'I\'d be happy to help you solve this equation step by step! Let\'s work through it together.\n\nYou have the equation: 3x + 8 = 20\n\nThe goal is to get x by itself on one side of the equation. \n\nFirst, let\'s think about what we need to do to isolate the term with x. What do you think should be our first step to get rid of the "+8" on the left side?'

In [ ]:
# system prompt exercise

messages = []

add_user_message(messages, "Write a Python function that checks a string for duplicate characters.")
system = """
    You are a Python engineer who writes very concise code.
"""
add_system_message(messages, system)
answer = chat(messages)   
answer

ProxyError: HTTPSConnectionPool(host='api.laozhang.ai', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by ProxyError('Unable to connect to proxy', SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:997)'))))

In [6]:
messages = []

add_user_message(messages, "Generate a one sentence movie idea.")
answer = chat(messages, temperature=0.9)
answer

"A janitor at a memory clinic discovers they can experience patients' forgotten memories through objects left behind, but becomes dangerously addicted to living other people's lives."

In [19]:
# streaming example
import json

messages = []
add_user_message(messages, "Generate a 1 sentence description of a NoSQL database.")

params = {
        "model": model,
        "stream": True,
        "messages": messages,
        "max_tokens": 1024,
}


response = requests.post(url, headers=headers, json=params)
for line in response.iter_lines():
    if line:
        line = line.decode('utf-8')
        if line.startswith('data: '):
            data = line[6:]
            if data != '[DONE]':
                chunk = json.loads(data)
                content = chunk['choices'][0]['delta'].get('content', '')
                print(content, end='')

# ##########################################
# streaming with official anthropic library
# ##########################################

# from anthropic import Anthropic
# client = Anthropic(
#     api_key=api_key
# )

# with client.messages.stream(
#     model=model,
#     messages=[
#         {"role": "user", "content": "Generate a 1 sentence description of a NoSQL database."}
#     ],
#     max_tokens=1024,
# ) as stream:
#     for text in stream.text_stream:
#         print(text, end='')
# stream.get_final_message()

NoSQL databases are non-relational data storage systems that use flexible schemas to handle large volumes of unstructured or semi-structured data across distributed computing environments.

In [20]:
# Prefilled & Stop

messages = []
add_user_message(messages, "Count 1 to 10")
answer = chat(messages,stop_sequences=["5"])
print(answer)

Here's counting from 1 to 10:

1
2
3
4



In [24]:
# structured data
messages = []

add_user_message(messages, "Generate a very short event bridge rule as json.")
add_assistant_message(messages, "```json")
text = chat(messages,stop_sequences=["```"])

json.loads(text.strip())

{'Name': 'OrderProcessingRule',
 'EventPattern': {'source': ['myapp.orders'], 'detail-type': ['Order Placed']},
 'Targets': [{'Id': '1',
   'Arn': 'arn:aws:lambda:us-east-1:123456789012:function:ProcessOrder'}]}